# Product Recommendation Model with GPU Training (Workspaces)

This notebook demonstrates GPU-accelerated ML model training in **Snowflake Notebooks in Workspaces**.

**Use Case:** Predict which products a customer is likely to purchase next based on their order history

**What we'll do:**
1. Set execution context for Workspaces
2. Load customer order data from Snowflake
3. Engineer features from purchase patterns
4. Train XGBoost model with GPU acceleration
5. Evaluate model performance
6. Save model to Snowflake Model Registry

**Prerequisites:**
- Run in Snowflake Notebooks in Workspaces with GPU compute pool
- Data in AUTOMATED_INTELLIGENCE.RAW.ORDERS and ORDER_ITEMS

## 1. Set Execution Context

Notebooks in Workspaces don't automatically set database or schema. We need to explicitly set the execution context.

In [ ]:
%%sql
-- Set execution context for Workspaces
USE DATABASE AUTOMATED_INTELLIGENCE;
USE SCHEMA RAW;
USE WAREHOUSE COMPUTE_WH;  -- Adjust to your warehouse name

-- Verify context
SELECT CURRENT_DATABASE() as database, 
       CURRENT_SCHEMA() as schema, 
       CURRENT_WAREHOUSE() as warehouse,
       CURRENT_ROLE() as role;

## 2. Check GPU Availability

Verify that GPU is available in the compute pool.

In [ ]:
import subprocess

# Check GPU with nvidia-smi
try:
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
        capture_output=True, 
        text=True, 
        timeout=5
    )
    if result.returncode == 0:
        gpu_info = result.stdout.strip()
        print(f"🚀 GPU Detected: {gpu_info}")
        GPU_AVAILABLE = True
    else:
        print("⚠️  No GPU detected - will use CPU training")
        GPU_AVAILABLE = False
except Exception as e:
    print(f"⚠️  Could not detect GPU: {e}")
    print("   Will use CPU training")
    GPU_AVAILABLE = False

print(f"\n{'GPU' if GPU_AVAILABLE else 'CPU'} training will be used")

## 3. Import Libraries

Import required libraries for data processing, ML, and Snowflake integration.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    classification_report, 
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score
)
import xgboost as xgb
import datetime
import warnings

warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")
print(f"   XGBoost version: {xgb.__version__}")
print(f"   Pandas version: {pd.__version__}")

## 4. Load and Explore Data

Query orders and order items using fully qualified table names.

In [ ]:
%%sql
-- Check data volume with fully qualified names
SELECT 
    'ORDERS' as table_name,
    COUNT(*) as row_count,
    COUNT(DISTINCT CUSTOMER_ID) as unique_customers
FROM AUTOMATED_INTELLIGENCE.RAW.ORDERS
UNION ALL
SELECT 
    'ORDER_ITEMS' as table_name,
    COUNT(*) as row_count,
    COUNT(DISTINCT ORDER_ID) as unique_orders
FROM AUTOMATED_INTELLIGENCE.RAW.ORDER_ITEMS;

In [ ]:
%%sql
-- Sample orders
SELECT 
    ORDER_ID, 
    CUSTOMER_ID, 
    ORDER_DATE, 
    TOTAL_AMOUNT, 
    ORDER_STATUS
FROM AUTOMATED_INTELLIGENCE.RAW.ORDERS
LIMIT 10;

In [ ]:
%%sql
-- Sample order items
SELECT 
    ORDER_ID, 
    PRODUCT_ID, 
    QUANTITY, 
    UNIT_PRICE, 
    LINE_TOTAL
FROM AUTOMATED_INTELLIGENCE.RAW.ORDER_ITEMS
LIMIT 10;

## 5. Feature Engineering

Create customer and product features for recommendation model.

In [ ]:
%%sql
-- Create training dataset with customer-product features
CREATE OR REPLACE TEMPORARY TABLE training_features AS
WITH customer_stats AS (
    SELECT 
        o.CUSTOMER_ID,
        COUNT(DISTINCT o.ORDER_ID) as TOTAL_PURCHASES,
        SUM(oi.LINE_TOTAL) as TOTAL_SPENT,
        AVG(oi.LINE_TOTAL) as AVG_ITEM_SPEND,
        COUNT(DISTINCT oi.PRODUCT_ID) as UNIQUE_PRODUCTS,
        MAX(o.ORDER_DATE) as LAST_PURCHASE_DATE,
        DATEDIFF(day, MAX(o.ORDER_DATE), CURRENT_DATE()) as DAYS_SINCE_LAST_PURCHASE
    FROM AUTOMATED_INTELLIGENCE.RAW.ORDERS o
    JOIN AUTOMATED_INTELLIGENCE.RAW.ORDER_ITEMS oi 
        ON o.ORDER_ID = oi.ORDER_ID
    GROUP BY o.CUSTOMER_ID
),
customer_products AS (
    SELECT 
        o.CUSTOMER_ID,
        oi.PRODUCT_ID,
        COUNT(*) as PURCHASE_COUNT,
        SUM(oi.QUANTITY) as TOTAL_QUANTITY,
        MAX(o.ORDER_DATE) as LAST_PRODUCT_PURCHASE
    FROM AUTOMATED_INTELLIGENCE.RAW.ORDERS o
    JOIN AUTOMATED_INTELLIGENCE.RAW.ORDER_ITEMS oi 
        ON o.ORDER_ID = oi.ORDER_ID
    GROUP BY o.CUSTOMER_ID, oi.PRODUCT_ID
)
SELECT 
    cp.CUSTOMER_ID,
    cp.PRODUCT_ID,
    cs.TOTAL_PURCHASES,
    cs.TOTAL_SPENT,
    cs.AVG_ITEM_SPEND,
    cs.UNIQUE_PRODUCTS,
    cs.DAYS_SINCE_LAST_PURCHASE,
    cp.PURCHASE_COUNT,
    cp.TOTAL_QUANTITY,
    1 as PURCHASED  -- Label: customer purchased this product
FROM customer_products cp
JOIN customer_stats cs ON cp.CUSTOMER_ID = cs.CUSTOMER_ID
-- Sample for faster training (adjust as needed)
SAMPLE (100000 ROWS);

-- Check results
SELECT COUNT(*) as training_examples FROM training_features;

In [ ]:
%%sql
-- Preview features
SELECT * FROM training_features LIMIT 10;

## 6. Load Data into Pandas for Training

Reference the SQL result using cell referencing.

In [ ]:
%%sql
-- Load all training data
SELECT * FROM training_features;

In [ ]:
# Reference SQL cell result (assuming previous cell is dataframe_1)
# Note: Adjust dataframe_X number based on actual cell execution order
pdf = dataframe_1.copy()  # Replace with actual dataframe number

print(f"✅ Loaded {len(pdf):,} training examples")
print(f"\n📊 Data Shape: {pdf.shape}")
print(f"\n📈 Feature Summary:")
print(pdf.describe())

## 7. Encode Categorical Features

In [ ]:
# Encode customer_id and product_id
customer_encoder = LabelEncoder()
product_encoder = LabelEncoder()

pdf['CUSTOMER_ID_ENCODED'] = customer_encoder.fit_transform(pdf['CUSTOMER_ID'])
pdf['PRODUCT_ID_ENCODED'] = product_encoder.fit_transform(pdf['PRODUCT_ID'])

# Define features for training
feature_columns = [
    'CUSTOMER_ID_ENCODED',
    'PRODUCT_ID_ENCODED',
    'TOTAL_PURCHASES',
    'TOTAL_SPENT',
    'AVG_ITEM_SPEND',
    'UNIQUE_PRODUCTS',
    'DAYS_SINCE_LAST_PURCHASE',
    'PURCHASE_COUNT',
    'TOTAL_QUANTITY'
]

label_column = 'PURCHASED'

# Prepare X and y
X = pdf[feature_columns]
y = pdf[label_column]

print(f"📊 Features shape: {X.shape}")
print(f"   Label distribution: {y.value_counts().to_dict()}")
print(f"\n✅ Features encoded successfully")

## 8. Train XGBoost Model with GPU Acceleration

**GPU Training Configuration:**
- `tree_method='gpu_hist'`: Use GPU-accelerated histogram-based training
- `predictor='gpu_predictor'`: Use GPU for predictions
- Significantly faster than CPU for large datasets

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"📊 Data Split:")
print(f"   Training set: {len(X_train):,} samples")
print(f"   Test set: {len(X_test):,} samples")

In [ ]:
import time

# Configure XGBoost for GPU training
if GPU_AVAILABLE:
    print("🚀 Training with GPU acceleration...")
    xgb_params = {
        'n_estimators': 200,
        'max_depth': 8,
        'learning_rate': 0.1,
        'tree_method': 'gpu_hist',  # GPU-accelerated training
        'predictor': 'gpu_predictor',  # GPU-accelerated prediction
        'random_state': 42,
        'eval_metric': 'logloss'
    }
else:
    print("💻 Training with CPU...")
    xgb_params = {
        'n_estimators': 200,
        'max_depth': 8,
        'learning_rate': 0.1,
        'tree_method': 'hist',  # CPU histogram-based training
        'random_state': 42,
        'eval_metric': 'logloss',
        'n_jobs': -1  # Use all CPU cores
    }

# Create and train model
model = xgb.XGBClassifier(**xgb_params)

print(f"\n⏳ Training started...")
start_time = time.time()

model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=50  # Print progress every 50 iterations
)

training_time = time.time() - start_time

print(f"\n✅ Training complete!")
print(f"   Training time: {training_time:.2f} seconds")
print(f"   Model: XGBoost Classifier")
print(f"   Features: {len(feature_columns)}")
print(f"   Training samples: {len(X_train):,}")

## 9. Evaluate Model Performance

In [ ]:
# Make predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Classification report
print("📊 Model Performance:")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Not Purchased', 'Purchased']))

# Calculate accuracy metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"\n📈 Summary Metrics:")
print(f"   Accuracy: {accuracy:.4f}")
print(f"   Precision: {precision:.4f}")
print(f"   Recall: {recall:.4f}")
print(f"   F1 Score: {f1:.4f}")

# Business interpretation
print(f"\n💼 Business Impact:")
print(f"   If we recommend products to customers:")
print(f"   - {precision*100:.1f}% of recommendations will be relevant")
print(f"   - We'll capture {recall*100:.1f}% of products customers want")

## 10. Feature Importance Analysis

In [ ]:
# Get feature importance
feature_importance = model.feature_importances_

# Create DataFrame for visualization
feature_importance_df = pd.DataFrame({
    'feature': feature_columns,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("🔍 Feature Importance:")
print(feature_importance_df.to_string(index=False))

print("\n💡 Key Insights:")
top_3_features = feature_importance_df.head(3)
for idx, row in top_3_features.iterrows():
    print(f"   {row['feature']}: {row['importance']:.2%}")

## 11. Save Model to Snowflake Model Registry

**Note:** Requires Snowflake ML package and appropriate permissions.

In [ ]:
# Import Snowflake ML registry
from snowflake.ml.registry import Registry
from snowflake.snowpark import Session

# Create Snowpark session for Workspaces
# In Workspaces, connection is handled by the service
import snowflake.connector
import os

# Get connection from environment (Workspaces provides this)
conn = snowflake.connector.connect(
    account=os.environ.get('SNOWFLAKE_ACCOUNT'),
    user=os.environ.get('SNOWFLAKE_USER'),
    authenticator='externalbrowser'  # Use SSO if available
)

# Create Snowpark session
from snowflake.snowpark import Session
session = Session.builder.configs({'connection': conn}).create()

print("✅ Snowpark session created")
print(f"   Current database: {session.get_current_database()}")
print(f"   Current schema: {session.get_current_schema()}")

In [ ]:
# Create registry
registry = Registry(
    session=session,
    database_name="AUTOMATED_INTELLIGENCE",
    schema_name="MODELS"
)

# Prepare model metadata
model_name = "product_recommendation_xgboost"
version_name = f"v_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"

# Sample input for schema inference
sample_input = X_test.head(5)

# Save model
print(f"💾 Saving model to registry...")
print(f"   Model: {model_name}")
print(f"   Version: {version_name}")

try:
    mv = registry.log_model(
        model=model,
        model_name=model_name,
        version_name=version_name,
        sample_input_data=sample_input,
        comment=f"XGBoost product recommender {'(GPU-trained)' if GPU_AVAILABLE else '(CPU-trained)'}. Trained on {len(X_train):,} customer-product pairs. F1: {f1:.4f}",
        metrics={
            "accuracy": float(accuracy),
            "precision": float(precision),
            "recall": float(recall),
            "f1_score": float(f1),
            "training_samples": int(len(X_train)),
            "test_samples": int(len(X_test)),
            "training_time_seconds": float(training_time),
            "gpu_accelerated": GPU_AVAILABLE
        }
    )

    print(f"\n✅ Model saved successfully!")
    print(f"\n📦 Model Details:")
    print(f"   Database: AUTOMATED_INTELLIGENCE")
    print(f"   Schema: MODELS")
    print(f"   Model: {model_name}")
    print(f"   Version: {version_name}")
    print(f"   GPU Accelerated: {GPU_AVAILABLE}")
    print(f"   Training Time: {training_time:.2f}s")
except Exception as e:
    print(f"⚠️  Error saving model: {e}")
    print("   Check that AUTOMATED_INTELLIGENCE.MODELS schema exists and you have write permissions")

## 12. Test Model Predictions

Generate product recommendations for sample customers.

In [ ]:
# Get predictions for test set
X_sample = X_test.head(20)
predictions_proba = model.predict_proba(X_sample)[:, 1]

# Create results DataFrame
results = X_sample.copy()
results['PURCHASE_PROBABILITY'] = predictions_proba
results['PREDICTION'] = (predictions_proba > 0.5).astype(int)
results['ACTUAL'] = y_test.iloc[:20].values

# Sort by probability
results = results.sort_values('PURCHASE_PROBABILITY', ascending=False)

print("🔮 Top Product Recommendations:")
print(results[['CUSTOMER_ID_ENCODED', 'PRODUCT_ID_ENCODED', 
               'PURCHASE_PROBABILITY', 'PREDICTION', 'ACTUAL']].to_string(index=False))

# Calculate recommendation accuracy
correct = (results['PREDICTION'] == results['ACTUAL']).sum()
total = len(results)
print(f"\n✅ Recommendation Accuracy: {correct}/{total} ({correct/total*100:.1f}%)")

## Summary

**What we accomplished:**
- ✅ Set execution context for Notebooks in Workspaces
- ✅ Loaded customer order data using fully qualified table names
- ✅ Engineered features from purchase patterns
- ✅ Trained XGBoost model with GPU acceleration (if available)
- ✅ Achieved good prediction accuracy for product recommendations
- ✅ Saved model to Snowflake Model Registry

**Key Differences in Workspaces:**
- Explicit database/schema setting with `USE` statements
- Fully qualified table names for portability
- Service-based compute with long-running kernels
- Terminal access for package management
- IPython kernel with Jupyter magics support

**GPU Benefits:**
- Faster training on large datasets
- Can handle more trees and deeper models
- Same prediction accuracy as CPU

**Next Steps:**
1. Deploy model for real-time recommendations
2. A/B test recommendations in production
3. Monitor model performance over time
4. Retrain periodically with new data